In [2]:
import pandas as pd
#Es una clase de búsqueda de cuadrícula que ayuda a encontrar los mejores hiperparámetros para un algoritmo de recomendación utilizando validación cruzada.
from surprise.model_selection import GridSearchCV, train_test_split
'''
surprise.SVD: Es un algoritmo de factorización matricial para sistemas de recomendación implementado en la biblioteca Surprise.
surprise.Dataset: Es una clase que representa un conjunto de datos en formato adecuado para su uso con algoritmos de recomendación de Surprise.
surprise.Reader: Es una clase utilizada para especificar el formato de los datos de entrada.
surprise.accuracy: Es un módulo que proporciona funciones para calcular métricas de precisión para los modelos de recomendación.
'''
from surprise import SVD, Dataset, Reader, accuracy
import pickle

In [3]:
games=pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\games.parquet')
items=pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\items.parquet')
reviews=pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\reviews.parquet')
final=pd.read_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\data_final.parquet')

In [4]:
print(reviews.head())
print(reviews.shape)

             user_id item_id  recommend  sentiment_analysis
0  76561197970982479    1250       True                   1
0  76561197970982479   22200       True                   2
0  76561197970982479   43110       True                   1
1            js41637  251610       True                   1
1            js41637  227300       True                   1
(59305, 4)


In [5]:
# Reemplazar los valores booleanos en la columna 'recommend' con valores numéricos

final['recommend'].replace({True:1,False:0}, inplace=True)


C:\Users\Moises\AppData\Local\Temp\ipykernel_8884\3376326048.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final['recommend'].replace({True:1,False:0}, inplace=True)
C:\Users\Moises\AppData\Local\Temp\ipykernel_8884\3376326048.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final['recommen

In [6]:
# Crear una nueva columna 'Rating' que sea la suma de 'recommend' y 'sentiment_analysis'

final['Ratting']=final['recommend']+final['sentiment_analysis']

In [7]:
final.head()

,user_id,item_id,recommend,sentiment_analysis,genres,title,release_date,price,developer,items_count,playtime_forever,playtime_2weeks,Ratting
0,76561197970982479,1250,1,1,Action,Killing Floor,2009,19.99,Tripwire Interactive,277,6.0,0.0,2
1,76561197970982479,1250,1,1,Action,Killing Floor,2009,19.99,Tripwire Interactive,277,0.0,0.0,2
2,76561197970982479,1250,1,1,Action,Killing Floor,2009,19.99,Tripwire Interactive,277,7.0,0.0,2
3,76561197970982479,1250,1,1,Action,Killing Floor,2009,19.99,Tripwire Interactive,277,4733.0,0.0,2
4,76561197970982479,1250,1,1,Action,Killing Floor,2009,19.99,Tripwire Interactive,277,1853.0,0.0,2


In [8]:
# Crear un nuevo DataFrame 'prueba' con las columnas 'user_id', 'title' y 'Rating' de 'final'

prueba=final[['user_id','title','Ratting']]

In [9]:
prueba.head()

,user_id,title,Ratting
0,76561197970982479,Killing Floor,2
1,76561197970982479,Killing Floor,2
2,76561197970982479,Killing Floor,2
3,76561197970982479,Killing Floor,2
4,76561197970982479,Killing Floor,2


In [10]:
# Crear una muestra aleatoria del 5% de las filas del DataFrame 'prueba'

muestra=prueba.sample(frac=0.05, random_state=42)

In [11]:
# Crear un objeto Reader con una escala de calificación personalizada de 0 a 3
reader=Reader(rating_scale=(0, 3)) 
# Cargar los datos del DataFrame muestra en un objeto Dataset utilizando el formato especificado por el objeto Reader
data=Dataset.load_from_df(muestra,reader)

In [12]:
# Definir el conjunto de parámetros para la búsqueda de cuadrícula
param_grid = {
    'n_factors': [0, 100], # Número de factores latentes
    'n_epochs': [0, 50], # Número de épocas de entrenamiento
    'lr_all': [0, 0.01], # Tasa de aprendizaje para todos los parámetros
    'reg_all': [0, 0.05], # Término de regularización para todos los parámetros
    'random_state': [0, 42] # Semilla aleatoria para reproducibilidad
}
# Inicializar el modelo SVD
model = SVD()
# Inicializar la búsqueda de cuadrícula
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
# Realizar la búsqueda de cuadrícula en los datos
grid_search.fit(data)
# Obtener los mejores parámetros encontrados
best_params = grid_search.best_params['rmse']
# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:", best_params)

Mejores parámetros encontrados: {'n_factors': 100, 'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0, 'random_state': 0}


In [13]:
# Cargar los datos del DataFrame 'prueba' en un objeto Dataset utilizando el formato especificado por el objeto Reader
data_prueba=Dataset.load_from_df(prueba,reader)

In [14]:
from surprise import accuracy
# Dividir los datos en conjuntos de entrenamiento y prueba
trainset, testset=train_test_split(data, test_size=0.2)
# Inicializar el modelo SVD con los mejores hiperparámetros encontrados
model=SVD(n_factors=100,n_epochs=50,lr_all=0.01,reg_all=0,random_state=42)
# Entrenar el modelo en el conjunto de entrenamiento
model.fit(trainset)
# Hacer predicciones en el conjunto de prueba
predictions=model.test(testset)
# Calcular el error cuadrático medio de raíz (RMSE)
rmse=accuracy.rmse(predictions)
# Imprimir el RMSE
print(f"RMSE: {rmse}")

RMSE: 0.0839
RMSE: 0.08389298489708154


In [15]:
def recomendaciones_usuario(usuario: str):
    if usuario not in prueba['user_id'].unique():
        # Si el usuario no está en la lista de usuarios del conjunto de datos
        # Se seleccionan aleatoriamente 5 juegos para recomendar
        juegos_aleatorios = list(prueba['title'].sample(5))
        mensaje = f'El usuario {usuario} no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria.'
        return {mensaje: juegos_aleatorios}
    else:
        diccionario = {}
        
        # Se obtienen los juegos valorados por el usuario
        juegos_valorados = prueba[prueba['user_id'] == usuario]['title'].unique()
        todos_los_juegos = prueba['title'].unique()
        
        # Se encuentran los juegos no valorados por el usuario
        juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

        # Se generan predicciones para los juegos no valorados
        predicciones = [model.predict(usuario, juego) for juego in juegos_no_valorados]
        
        # Se seleccionan las 5 mejores recomendaciones basadas en las predicciones
        recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5] 
        cont = 1
        for recomendacion in recomendaciones:
            # Se crea un diccionario de las 5 mejores recomendaciones
            diccionario[f'Top {cont}'] = recomendacion.iid
            cont += 1
            
        return {f'Usuario: {usuario}': diccionario}

In [16]:
testeo1 = recomendaciones_usuario('Freejam')
testeo1

{'El usuario Freejam no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria.': ['Goat Simulator',
  'StarDrive',
  'The Stanley Parable',
  'DayZ',
  'Clicker Heroes']}

In [17]:
usuarios = prueba['user_id'].unique() # --- > Testeo diferentes usuarios para comprobar su comportamiento

diccionario = {}

for x in usuarios[:5]:
    
    diccionario[x] = recomendaciones_usuario(x)

diccionario

{'76561197970982479': {'Usuario: 76561197970982479': {'Top 1': "Garry's Mod",
   'Top 2': 'Rocket League®',
   'Top 3': 'Team Fortress 2',
   'Top 4': 'Choice of Robots',
   'Top 5': 'Trine Enchanted Edition'}},
 'js41637': {'Usuario: js41637': {'Top 1': "King Arthur's Gold",
   'Top 2': 'Realm of the Mad God',
   'Top 3': 'PlanetSide 2',
   'Top 4': 'Homeworld Remastered Collection',
   'Top 5': 'Company of Heroes 2'}},
 'evcentric': {'Usuario: evcentric': {'Top 1': 'Team Fortress 2',
   'Top 2': "Sid Meier's Civilization® V",
   'Top 3': 'Just Cause 2: Multiplayer Mod',
   'Top 4': 'The Walking Dead: Season 2',
   'Top 5': 'The Wolf Among Us'}},
 'doctr': {'Usuario: doctr': {'Top 1': 'Assassin’s Creed® IV Black Flag™',
   'Top 2': 'X-Plane 10 Global - 64 Bit',
   'Top 3': 'Euro Truck Simulator 2',
   'Top 4': 'Hyperdimension Neptunia Re;Birth1 / 超次次元ゲイム ネプテューヌRe;Birth1 / 超次次元遊戲戰機少女重生1',
   'Top 5': 'Team Fortress 2'}},
 'maplemage': {'Usuario: maplemage': {'Top 1': 'Crysis 2 - Maximu

In [18]:
#lo convierto a parquet
prueba.to_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\Relacionamiento.parquet',index=False)

In [19]:
#y ahora lo guardo como pkl
with open (r'C:\Users\Moises\Desktop\Projecto invidual 1\03Modelos\Relacionamiento.pkl', 'wb') as file:
    pickle.dump(model,file)